In [1]:
#imports
import numpy as np

In [55]:
#load small input
input = open("./../../data/day_23/small_input.txt", "r").readlines()

In [78]:
#load input
input = open("./../../data/day_23/input.txt", "r").readlines()

In [79]:
#process input
layout = np.zeros((len(input), len(input[0].strip())), dtype=np.int8)
for i,line in enumerate(input):
    for j,char in enumerate(line.strip()):
        match char:
            case "#": layout[i,j] = 1
            case ".":
                if i == 0:
                    start_position = (0,j)
                elif i == len(input)-1:
                    end_position = (len(input)-1,j)
            case ">": layout[i,j] = 2
            case "v": layout[i,j] = 3
            case "<": layout[i,j] = 4
            case "^": layout[i,j] = 5

In [80]:
#part 1
def get_neighbours(pos,shape,cont = None):
    neighbours = []
    x,y = pos
    xm,ym = shape
    if cont == 0: cont = None
    if cont is None:
        if x > 0:
            neighbours.append((x-1,y))
        if y > 0:
            neighbours.append((x,y-1))
        if x < xm-1:
            neighbours.append((x+1,y))
        if y < ym-1:
            neighbours.append((x,y+1))
    else:
        match cont:
            case 2:
                if y < ym-1:
                    neighbours.append((x,y+1))
            case 3:
                if x < xm-1:
                    neighbours.append((x+1,y))
            case 4:
                if y > 0:
                    neighbours.append((x,y-1))
            case 5:
                if x > 0:
                    neighbours.append((x-1,y))
                
    return neighbours

cache_neighbours = {}
for x in range(layout.shape[0]):
    for y in range(layout.shape[1]):
        cache_neighbours[(x,y)] = get_neighbours((x,y),layout.shape,cont=layout[x,y])



path_lengths_to_target = []
def find_longest_path(start,target,layout,cache_neighbours, visited, out):
    if start == target:
        return
    visited[start] = True
    neighbours = cache_neighbours[start]
    for n in neighbours:
        if layout[n] == 1:
            continue
        if n == target:
            out.append(len(visited))
            continue
        if n not in visited:
            find_longest_path(n,target,layout,cache_neighbours, {k:v for k,v in visited.items()}, out)
        #simply continue if the position was already visited
            
find_longest_path(start_position,end_position,layout,cache_neighbours,{},path_lengths_to_target)

result = max(path_lengths_to_target)
print(f"Part 1: {result}")



Part 1: 2306


In [58]:
# part 2
import sys
sys.setrecursionlimit(100000)
##remap layout
layout2 = np.copy(layout)
layout2[layout2 >= 2] = 0

cache_neighbours2 = {}
for x in range(layout2.shape[0]):
    for y in range(layout2.shape[1]):
        cache_neighbours2[(x,y)] = get_neighbours((x,y),layout2.shape,cont=layout2[x,y])

path_lengths_to_target2 = []
find_longest_path(start_position,end_position,layout2,cache_neighbours2,{},path_lengths_to_target2)

result2 = max(path_lengths_to_target2)
print(f"Part 2: {result2}")

Part 2: 154


In [84]:
##improve speed by lowering search space

layout2 = np.copy(layout)
layout2[layout2 >= 2] = 0

cache_neighbours2 = {}
for x in range(layout2.shape[0]):
    for y in range(layout2.shape[1]):
        neighbours = get_neighbours((x,y),layout2.shape,cont=layout2[x,y])
        valid_neighbours = [n for n in neighbours if layout2[n]==0]
        cache_neighbours2[(x,y)] = valid_neighbours




#find all intersections
intersections = {}
for x in range(layout2.shape[0]):
    for y in range(layout2.shape[1]):
        neighbours = cache_neighbours2[(x,y)]
        if len(neighbours) > 2:
            intersections[(x,y)] = neighbours
intersections[start_position] = cache_neighbours2[start_position]
intersections[end_position] = cache_neighbours2[end_position]

#find the lengths of the direct paths between connected intersections
intersection_connections =  {k:{} for k in intersections}
for start in intersections:
    for n in cache_neighbours2[start]:
        visited = {start: True}
        q = [n]
        while q:
            current = q.pop(0)
            visited[current] = True
            if current in intersections:
                intersection_connections[start][current] = len(visited)-1
                intersection_connections[current][start] = len(visited)-1
                break
            else:
                neighbours = cache_neighbours2[current]
                for n in neighbours:
                    if n not in visited:
                        q.append(n)

# for k,v in intersection_connections.items():
#     for k2,v2 in v.items():
#         print(f"{k} -> {k2}: {v2}")

def find_longest_path2(start,target,layout,intersection_connections, visited, out):
    if start == target:
        return
    visited[start] = True
    neighbours = intersection_connections[start].keys()
    for n in neighbours:
        if n == target:
            path = list(visited.keys())
            path.append(target)
            out.append(path)
            ##output because of long exec time
            path_length = 0

            continue
        if n not in visited:
            find_longest_path2(n,target,layout,intersection_connections, {k:v for k,v in visited.items()}, out)
        #simply continue if the position was already visited
            

paths_to_target = []
find_longest_path2(start_position,end_position,layout2,intersection_connections,{},paths_to_target)

path_lengths_to_target2 = []
for path in paths_to_target:
    path_length = 0
    for i in range(len(path)-1):
        p1,p2 = path[i],path[i+1]
        dist = intersection_connections[p1][p2]
        path_length += dist
    path_lengths_to_target2.append(path_length)

result2 = max(path_lengths_to_target2) #remove the start and end position
print(f"Part 2: {result2}")
    



Part 2: 6718
